In [2]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
!pip install transformers
!pip install sentencepiece
import torch
from transformers import TrainingArguments, Trainer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import plot_confusion_matrix
import numpy as np
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification  
import pandas as pd


     |████████████████████████████████| 2.1MB 5.7MB/s 
     |████████████████████████████████| 901kB 26.8MB/s 
     |████████████████████████████████| 3.3MB 34.4MB/s 
     |████████████████████████████████| 1.2MB 5.1MB/s 


In [4]:
# tokenizer = AutoTokenizer.from_pretrained("joeddav/xlm-roberta-large-xnli")
# model = AutoModelForSequenceClassification.from_pretrained("joeddav/xlm-roberta-large-xnli")

In [5]:
# classes = ["NOT_oFF", "OFF"]
df_test = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/test.xlsx')
df_test_labels = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/offLabel.xlsx')

#Label Encoding to convert labels to integer values
y_test = df_test_labels.labels
LE = LabelEncoder()
y_test=LE.fit_transform(y_test)
print(y_test[:20])
# pred1 = []
# df_test_labels.head
# df_test.head

[1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0]


In [6]:
from transformers import pipeline
Model='typeform/roberta-large-mnli'

classifier = pipeline("zero-shot-classification", model=Model,tokenizer=Model)

404 Client Error: Not Found for url: https://huggingface.co/typeform/roberta-large-mnli/resolve/main/pytorch_model.bin


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at typeform/roberta-large-mnli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [7]:
y_pred_new=[]
for tweet in df_test.tweets:
    pred_zero=classifier(tweet, ["toxic", "not_toxic"])
    # pred_zero=list(pred_zero.items())
    # case = {pred_zero['sequence'] : tweet[0], pred_zero['scores']: tweet[1], pred_zero['labels']:tweet[2] }
    # case = {'key1': pred_zero['scores'][0], 'key2': pred_zero['scores'][1] }
    if(pred_zero["scores"][0]>= pred_zero["scores"][1]):
      y_pred_new.append(pred_zero["labels"][0])
    else:
      y_pred_new.append(pred_zero["labels"][1])
    # y_pred_new.append(int( ))
    SCORES = pred_zero["scores"]
    # CLASSESS = pred_zero["labels"]
    # y_pred_new = np.argmax(SCORES)
    # predicted_class = CLASSESS[BEST_INDEX]
    # y_pred_new.append(BEST_INDEX)
   
    # print(SCORES,'\n',pred_zero["labels"])

y_pred_new[:20]

['toxic',
 'not_toxic',
 'not_toxic',
 'not_toxic',
 'toxic',
 'toxic',
 'toxic',
 'toxic',
 'toxic',
 'not_toxic',
 'not_toxic',
 'toxic',
 'toxic',
 'not_toxic',
 'toxic',
 'toxic',
 'not_toxic',
 'toxic',
 'not_toxic',
 'not_toxic']

In [8]:
#Label Encoding to convert labels to integer values
y_pred=LE.fit_transform(y_pred_new)
print(y_pred[:20])
y_pred_new=1-y_pred
print(y_pred_new[:20])

[1 0 0 0 1 1 1 1 1 0 0 1 1 0 1 1 0 1 0 0]
[0 1 1 1 0 0 0 0 0 1 1 0 0 1 0 0 1 0 1 1]


In [9]:
print("Accuracy", accuracy_score(y_test, y_pred_new))

print("\nConfusion matrix\n",confusion_matrix(y_test, y_pred_new))
print("\nClassification report\n",classification_report(y_test, y_pred_new))

Accuracy 0.578

Confusion matrix
 [[1036  562]
 [ 282  120]]

Classification report
               precision    recall  f1-score   support

           0       0.79      0.65      0.71      1598
           1       0.18      0.30      0.22       402

    accuracy                           0.58      2000
   macro avg       0.48      0.47      0.47      2000
weighted avg       0.66      0.58      0.61      2000



In [ ]:
#  pred_zero=classifier('USER @USER  المسلمين والمسلمات شفاء لا يغادر سقما واصبغ عليهم عافيتك يا شافي يا معافى يا ارحم الراحمين.', ["hate", "not_hate"], device=0)
pred_zero=classifier(' #الدورة_الشهرية يا عديمين الرجولة يا الذكور يا اشباة الرجال يا عبيد الكورة الله يحرقكم', ["hate", "not_hate"], device=0)
    # pred_zero=list(pred_zero.items())
    # case = {pred_zero['sequence'] : tweet[0], pred_zero['scores']: tweet[1], pred_zero['labels']:tweet[2] }
    # case = {'key1': pred_zero['scores'][0], 'key2': pred_zero['scores'][1] }
SCORES = pred_zero["scores"]
labels=pred_zero["labels"]
y_pred = np.argmax(SCORES)
print(SCORES)
pred_zero["labels"]

[0.988742470741272, 0.011257510632276535]


['hate', 'not_hate']

# Experments

In [ ]:
# paraphrase = tokenizer(sequence_0, return_tensors="pt")
# 'not_paraphrase = tokenizer( sequence_1, return_tensors="pt")

In [ ]:
# # not_paraphrase_classification_logits = model(**not_paraphrase).logits
# 'paraphrase_classification_logits = model(**paraphrase).logits

In [ ]:
# # paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
# 'not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]


In [ ]:
# def comparelabel(label, predect):
#   if(label == predect):
#     return "success"
#   return "failed"

# for i in range(len(classes)):
# pred.append(int(paraphrase_results[0] < 0.5))
# pred.append(int(not_paraphrase_results[0] < 0.5))
# if(paraphrase_results[0] > 0.5):
#   print(f"{classes[0]}: {int(round(paraphrase_results[0] * 100))}% - label: {classes[label[0]]} - predict: {comparelabel(label[0],int(paraphrase_results[0] < 0.5))}")
# else:
#   print(f"{classes[1]}: {int(round(paraphrase_results[1] * 100))}% - label: {classes[label[0]]} - predict: {comparelabel(label[0],int(paraphrase_results[0] < 0.5))}")
# # print(f"{classes[i]}: {int(round(paraphrase_results[i] * 100))}% - label: {classes[label[k]]} - predict: {comparelabel()}")

In [ ]:
# for i in range(len(classes)):
#     print(f"{classes[i]}: {int(round(not_paraphrase_results[i] * 100))}%")

not: 75%
hate: 25%


In [ ]:
# df_test = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/test.xlsx')
# df_test_labels = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/testLabels.xlsx')

In [ ]:
# df_test_new=pd.concat([df_test, df_test_labels], axis=1)
# df_test_new=df_test_new.drop(columns=['id'])

# df_test_new

,tweets,label
0,@USER اما انت تقعد طول عمرك لا مبدا ولا راي ث...,HS
1,@USER @USER بتخاف نسوانك يزعلوا ولا ايه 😂 اه ...,NOT_HS
2,RT @USER: يا عـسانـى نـبـقى يا عـمري حـبايـب ...,NOT_HS
3,RT @USER: باقي البيان وينو ما شفنه يا برهان <...,NOT_HS
4,@USER @USER اللهم انت الشافي المعافي اشفيه وج...,NOT_HS
...,...,...
1995,RT @USER: الله لايوفقك يا مهند عسيري يا معوق ...,NOT_HS
1996,RT @USER: @USER حبيبي يا يوسف وانت طيب يا صاح...,NOT_HS
1997,RT @USER: يا بو محمد عشت يا طيب الفال<LF>عاشت...,NOT_HS
1998,أنا مستني الحلقة بقالي سنتين يا بضان يا ابن ا...,NOT_HS


In [ ]:
# grouped = df_test_new.groupby(df_test_new.label)
# y_test_HS = grouped.get_group("HS")[:50]
# y_test_Not_HS = grouped.get_group("NOT_HS")[:50]
# # y_test_HS=y_test_HS.label.values
# # y_test_Not_HS=y_test_Not_HS.label.values

# # print(y_test_HS.head())
# # print(y_test_Not_HS.head())
# # print('==========================================================================')
# print(' Hate Speech label shape | ',y_test_HS.shape)
# print(' Not Hate Speech label shape | ',y_test_Not_HS.shape)

# %load_ext google.colab.data_table
# y_test_HS


In [ ]:
# %load_ext google.colab.data_table
# y_test_Not_HS

In [ ]:
# # toknize_test_HS = tokenizer(y_test_HS.tweets.to_list(),   max_length=120,padding=True, truncation=True,return_tensors="pt")
# toknize_test_Not_HS = tokenizer(y_test_Not_HS.tweets.to_list(),   max_length=120,padding=True, truncation=True,return_tensors="pt")
# hs_classification_logits = model(**toknize_test_HS).logits
# not_hs_classification_logits = model(**toknize_test_Not_HS).logits
# import torch

# hs_results = torch.softmax(hs_classification_logits, dim=1)
# not_hs_results = torch.softmax(not_hs_classification_logits, dim=1)
# not_hs_results.tolist()

In [ ]:
# y_pred = np.argmax(not_hs_results, axis=1)

In [ ]:
# # Should be paraphrase
# classes = ["HS", "NOT-HS"]
# for i in range(len(classes)):
#     print(hs_results[i])
#     # print(f"{classes[i]}: {int(round(hs_results[i] * 100))}%")

In [ ]:
# for i in range(len(classes)):
#     print(not_hs_results[i])  
#     # print(f"{classes[i]}: {int( torch.round(not_hs_results[i] * 100))}%")

In [ ]:
# test_encodings=tokenizer(list(x_test),truncation=True,padding=True)
# test_encodings

In [ ]:
# # Create torch dataset
# class Dataset(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels=None):
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         if self.labels:
#             item["labels"] = torch.tensor(self.labels[idx])
#         return item

#     def __len__(self):
#         return len(self.encodings["input_ids"])

In [ ]:
# # Load test data
# # test_data = pd.read_csv("test.csv")
# X_test = list(df_test_new["tweets"])
# X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# # Create torch dataset
# test_dataset = Dataset(X_test_tokenized)

In [ ]:
# test_trainer=Trainer(model)

In [ ]:
# raw_pred = test_trainer.predict(test_dataset)
# raw_pred, _, _ = test_trainer.predict(test_dataset)

In [ ]:
# y_pred = np.argmax(raw_pred, axis=1)
# y_pred[:20]

In [ ]:
#Label Encoding to convert labels to integer values
# y_test=df_test_new.label
# LE = LabelEncoder()
# y_test=LE.fit_transform(y_test)
# y_test[:20]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1])

In [ ]:
# accuracy_score(y_test, y_pred)

0.364

In [ ]:
# accuracy = accuracy_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)

# print("accuracy", accuracy_score(y_test, y_pred))
# print("precision", precision)
# print( "recall", recall)
# print( "f1",f1)
# print('--------')
# print(classification_report(y_test,y_pred))
# print(confusion_matrix(y_test,y_pred))


accuracy 0.364
--------
              precision    recall  f1-score   support

           0       0.02      0.23      0.03       101
           1       0.90      0.37      0.53      1899

    accuracy                           0.36      2000
   macro avg       0.46      0.30      0.28      2000
weighted avg       0.86      0.36      0.50      2000

[[  23   78]
 [1194  705]]
